In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
dir_out = 'outputs/'

In [3]:
def xfeature(row, biotype):
	ret = 0
	for bio, con in zip(row['BIOTYPE'].split(','), row['Consequence'].split(',')):
		if bio == biotype and 'upstream_gene_variant' not in con and 'downstream_gene_variant' not in con:
			ret = 1
			break
	return ret

def xintergenic(row):
    ret = 0
    #cons_set_max = {'intergenic_variant', 'upstream_gene_variant', 'downstream_gene_variant'}
    cons_set_max = {'intergenic_variant'}
    cons_set = set(row['Consequence'].split(','))
    if (len(cons_set - cons_set_max) == 0) and (row['GENCODE'] == '.'):
        ret = 1
    return ret

def xgenic(row):
    ret = 0
    pli_all = []
    for p, con in zip(row['GENES_PLI'].split(','), row['Consequence'].split(',')):
        if (p == '.') or ('upstream_gene_variant' in con) or ('downstream_gene_variant' in con):
            continue
        pli_all.append(float(p))
    if len(pli_all) > 0:
        ret = 1
    return ret

### filtering INV calls which do not interupt genes
def xpli(row, pli_thr):
    ret = 0
    lof_inv_genes = row['LOF_DUP_INV_GENES'].split(',')
    for p, con, sym in zip(row['GENES_PLI'].split(','), row['Consequence'].split(','), row['SYMBOL'].split(',')):
        if (p == '.') or ('upstream_gene_variant' in con) or ('downstream_gene_variant' in con):
            continue
        if float(p) >= pli_thr and (row['SVTYPE']!='INV' or sym in lof_inv_genes):
            ret = 1
            break
    return ret

def xlowpli(row, pli_thr):
    ret = 0
    pli_all = []
    for p, con in zip(row['GENES_PLI'].split(','), row['Consequence'].split(',')):
        if (p == '.') or ('upstream_gene_variant' in con) or ('downstream_gene_variant' in con):
            continue
        pli_all.append(float(p))
    if len(pli_all) > 0 and max(pli_all) < pli_thr:
        ret = 1
    return ret

### filtering INV calls which do not interupt genes
def xloeuf(row, l_thr):
    ret = 0
    lof_inv_genes = row['LOF_DUP_INV_GENES'].split(',')
    for l, con, sym in zip(row['GENES_LOEUF'].split(','), row['Consequence'].split(','), row['SYMBOL'].split(',')):
        if (l == '.') or ('upstream_gene_variant' in con) or ('downstream_gene_variant' in con):
            continue
        if float(l) <= l_thr and (row['SVTYPE']!='INV' or sym in lof_inv_genes):
            ret = 1
            break
    return ret

### filtering INV calls which do not interupt genes
def xfdr(row, f_type, f_thr):
    ret = 0
    lof_inv_genes = row['LOF_DUP_INV_GENES'].split(',')
    for f, con, sym in zip(row['GENES_FDR_'+f_type].split(','), row['Consequence'].split(','), row['SYMBOL'].split(',')):
        if (f == '.') or ('upstream_gene_variant' in con) or ('downstream_gene_variant' in con):
            continue
        if float(f) <= f_thr and (row['SVTYPE']!='INV' or sym in lof_inv_genes):
            ret = 1
            break
    return ret

def xfb_pr_enh(row, fb_set):
    ret = 0
    if row['FB_PR_ENH_M'] != '.' and len( set(row['FB_PR_ENH_M'].split('&')).intersection(fb_set) ) != 0:
        ret = 1
    if row['FB_PR_ENH_F'] != '.' and len( set(row['FB_PR_ENH_F'].split('&')).intersection(fb_set) ) != 0:
        ret = 1
    return ret

def xgnocchi(row, col, gn_thr):
    ret = 0
    #if (row['GNOCCHI'] != '.') and (float(row['GNOCCHI']) > gn_thr):
    if (row[col] != '.') and (float(row[col]) > gn_thr):
        ret = 1
    return ret

def xfantom(row):
    ret = 0
    if row['FANTOM_ENH'] == 'FanEnh':
        ret = 1
    return ret
                
def status_num(row, cols):
    col_case_nums = []
    col_control_nums = []
    for c in cols:
        col = f'case_{c}'
        if row[col] == '.' or row[col] == '':
            col_case_nums.append(0)
            col_control_nums.append(0)
            continue
        n_case = len([x for x in row[col].split(',') if x == 'Yes'])
        n_control = len([x for x in row[col].split(',') if x == 'No'])
        col_case_nums.append(n_case)
        col_control_nums.append(n_control)
    return *col_case_nums, *col_control_nums
    
def xgencode(row, gen_list, feature):	
	ret = 0
	if row[feature] == 1 and row['GENCODE'] in (gen_list):
		ret = 1
	return ret

def x2features(row, feature1, feature2):
    ret = 0
    if row[feature1] == 1 and row[feature2] == 1:
        ret = 1
    return ret

def count_num(row, col):
    ret = 0
    if row[col] == '.' or row[col] == '':
        return ret
    ret = len(row[col].split(','))
    return ret

def lof(row):
    ret = 0
    if row['SVTYPE'] in ['DEL', 'INS'] and row['GENCODE'] in ['CDS', 'start_codon', 'stop_codon']:
        ret = 1
    if row['LOF_DUP_INV'] == 1:
        ret = 1
    return ret

def xdev_br(row, dev_br_list):
	ret = 0
	for sym, con in zip(row['SYMBOL'].split(','), row['Consequence'].split(',')):
		if (sym == '.') or ('upstream_gene_variant' in con) or ('downstream_gene_variant' in con):
			continue
		if sym in dev_br_list:
			ret = 1
			break
	return ret

def get_freq(row, col_lr, parents_set_lr, parents_set_sr):
    # also compute frequency of IL samples in addition to LR col
    if row['PLATFORM'] == 'LR_IL':
        #if row[col_lr] == '.' or row[col_lr] == '':
        #    return 0
        parents = [x for x in row[col_lr].split(',') if x in parents_set_lr]
        freq = float(f'{len(parents) / len(parents_set_lr):.4f}')
        return freq
    elif row['PLATFORM'] == 'LR':
        parents_lr = [x for x in row[col_lr].split(',') if x in parents_set_lr]
        freq_lr = float(f'{len(parents_lr) / len(parents_set_lr):.4f}')
        parents_il = [x for x in row['SV2_GT_SAMPLES'].split(',') if x in parents_set_lr]
        freq_il = float(f'{len(parents_il) / len(parents_set_lr):.4f}')
        freq = max(freq_lr, freq_il)
        return freq
    elif row['PLATFORM'] == 'IL':
        parents = [x for x in (row['HET_SAMPLES'].split(',') + row['HOMALT_SAMPLES'].split(',')) if x in parents_set_sr]
        freq = float(f'{len(parents) / len(parents_set_sr):.4f}')
        return freq
    else:
        return 'WRONG_FQ'

def xconserv(row, col, cons_thr):
    ret = 0
    if (row[col] != '.') and (float(row[col]) >= cons_thr):
        ret = 1
    return ret

def xsHet(row, s_thr):
	ret = 0
	for s, con in zip(row['GENES_S_HET'].split(','), row['Consequence'].split(',')):
		if (s == '.') or ('upstream_gene_variant' in con) or ('downstream_gene_variant' in con):
			continue
		if float(s) >= s_thr:
			ret = 1
			break
	return ret

def stratify_samples(row, col):
    samples_hp12 = []
    samples_ncom = []
    ad1_m_alt_samples = row['AD1_M_ALT_SAMPLES'].split(',')
    ad1_p_alt_samples = row['AD1_P_ALT_SAMPLES'].split(',')
    for sample in row[col].split(','):
        if (sample in ad1_m_alt_samples) and (sample in ad1_p_alt_samples):
            samples_hp12.append(sample)
        else:
            samples_ncom.append(sample)
    if len(samples_hp12) == 0:
        samples_hp12.append('.')
    if len(samples_ncom) == 0:
        samples_ncom.append('.')
    return ','.join(samples_hp12), ','.join(samples_ncom)

def platform_redef(row, col_plat_old, col_lr):
    plat_old = row[col_plat_old]
    if plat_old == 'LR_IL':
        return plat_old
    if plat_old == 'LR':
        if row['SV2_GT_SAMPLES'] != '.':
            return 'LR_IL'
        else:
            return 'LR'
    if plat_old == 'IL':
        if row[col_lr] != '.':
            return 'LR_IL'
        else:
            return 'IL'
    return 'WRONG'

In [4]:
def get_trans(row, col_lr, mom_kids_dict, dad_kids_dict, parent_kids_dict, mate_dict ,aff_dict):
    num_tran_case = 0
    num_tran_control = 0
    num_nontran_case = 0
    num_nontran_control = 0
    if row['PLATFORM'] != 'IL':
        moms = [x for x in row[col_lr].split(',') if x in mom_kids_dict]
        dads = [x for x in row[col_lr].split(',') if x in dad_kids_dict]
        mat_inh = [k for k in row['MAT_INH_LR_LC'].split(',')]
        pat_inh = [k for k in row['PAT_INH_LR_LC'].split(',')]
        ad1_m_ref_samples = row['AD1_M_REF_SAMPLES'].split(',')
        ad1_p_ref_samples = row['AD1_P_REF_SAMPLES'].split(',')
        for mom in moms:
            num_tran_case += len([k for k in mom_kids_dict[mom] if ((k in mat_inh) and (aff_dict[k] == 'Yes'))])
            num_tran_control += len([k for k in mom_kids_dict[mom] if ((k in mat_inh) and (aff_dict[k] == 'No'))])
            num_nontran_case += len([k for k in mom_kids_dict[mom] if ((k not in mat_inh) and (k in ad1_m_ref_samples) and (aff_dict[k] == 'Yes'))])
            num_nontran_control += len([k for k in mom_kids_dict[mom] if ((k not in mat_inh) and (k in ad1_m_ref_samples) and (aff_dict[k] == 'No'))])
        for dad in dads:
            num_tran_case += len([k for k in dad_kids_dict[dad] if ((k in pat_inh) and (aff_dict[k] == 'Yes'))])
            num_tran_control += len([k for k in dad_kids_dict[dad] if ((k in pat_inh) and (aff_dict[k] == 'No'))])
            num_nontran_case += len([k for k in dad_kids_dict[dad] if ((k not in pat_inh) and (k in ad1_p_ref_samples) and (aff_dict[k] == 'Yes'))])
            num_nontran_control += len([k for k in dad_kids_dict[dad] if ((k not in pat_inh) and (k in ad1_p_ref_samples) and (aff_dict[k] == 'No'))])
    else:
        parents = [x for x in row['IL_SAMPLES_LR'].split(',') if x in parent_kids_dict]
        het_samples = [x for x in row['HET_SAMPLES_LR'].split(',')]
        homalt_samples = [x for x in row['HOMALT_SAMPLES_LR'].split(',')]
        samples = het_samples + homalt_samples
        parent_proc = set()
        #tran_kids = [k for k in row['PMAT_INH_IL_LR'].split(',')]
        for parent in parents:
            mate = mate_dict[parent]
            if parent in parent_proc:
                continue
            parent_proc.update([parent, mate])
            if parent in homalt_samples and mate not in parents:
                for k in parent_kids_dict[parent]:
                    if k in het_samples:
                        if aff_dict[k] == 'Yes':
                            num_tran_case += 1
                        elif aff_dict[k] == 'No':
                            num_tran_control += 1
            elif parent in homalt_samples and mate in het_samples:
                for k in parent_kids_dict[parent]:
                    if k in het_samples:
                        if aff_dict[k] == 'Yes':
                            num_tran_case += 1
                            num_nontran_case += 1
                        elif aff_dict[k] == 'No':
                            num_tran_control += 1
                            num_nontran_control += 1
                    elif k in homalt_samples:
                        if aff_dict[k] == 'Yes':
                            num_tran_case += 2
                        elif aff_dict[k] == 'No':
                            num_tran_control += 2
            elif parent in homalt_samples and mate in homalt_samples:
                for k in parent_kids_dict[parent]:
                    if k in homalt_samples:
                        if aff_dict[k] == 'Yes':
                            num_tran_case += 2
                        elif aff_dict[k] == 'No':
                            num_tran_control += 2
            elif parent in het_samples and mate not in parents:
                for k in parent_kids_dict[parent]:
                    if k not in samples:
                        if aff_dict[k] == 'Yes':
                            num_nontran_case += 1
                        elif aff_dict[k] == 'No':
                            num_nontran_control += 1
                    elif k in het_samples:
                        if aff_dict[k] == 'Yes':
                            num_tran_case += 1
                        elif aff_dict[k] == 'No':
                            num_tran_control += 1
            elif parent in het_samples and mate in het_samples:
                for k in parent_kids_dict[parent]:
                    if k not in samples:
                        if aff_dict[k] == 'Yes':
                            num_nontran_case += 2
                        elif aff_dict[k] == 'No':
                            num_nontran_control += 2
                    elif k in het_samples:
                        if aff_dict[k] == 'Yes':
                            num_tran_case += 1
                            num_nontran_case += 1
                        elif aff_dict[k] == 'No':
                            num_tran_control += 1
                            num_nontran_control += 1
                    elif k in homalt_samples:
                        if aff_dict[k] == 'Yes':
                            num_tran_case += 2
                        elif aff_dict[k] == 'No':
                            num_tran_control += 2
            elif parent in het_samples and mate in homalt_samples:
                for k in parent_kids_dict[parent]:
                    if k in het_samples:
                        if aff_dict[k] == 'Yes':
                            num_tran_case += 1
                            num_nontran_case += 1
                        elif aff_dict[k] == 'No':
                            num_tran_control += 1
                            num_nontran_control += 1
                    elif k in homalt_samples:
                        if aff_dict[k] == 'Yes':
                            num_tran_case += 2
                        elif aff_dict[k] == 'No':
                            num_tran_control += 2
    return num_tran_case, num_tran_control, num_nontran_case, num_nontran_control

In [12]:
def get_big_table(file_name):
    df = pd.read_table(file_name, sep='\t', header=0)
    
    main_chroms = [f'chr{x}' for x in range(1,23)] + ['chrX', 'chrY']
    df = pd.DataFrame(df.loc[df.CHROM.isin(main_chroms)])

    ### to avoid recongizing NaN as float for empty entries
    df.LOF_DUP_INV_GENES = df.LOF_DUP_INV_GENES.fillna('')
    
    col_gns = ['GNOCCHI_MAX', 'GNOCCHI_MEAN']
    gn_thrs = [4, 3, 2, 1]
    for col_gn in col_gns:
        for gn_thr in gn_thrs:
            print(f'{col_gn}_{gn_thr}...')
            df[f'X_{col_gn}_{gn_thr}'] = df.apply(lambda row: xgnocchi(row, col_gn, gn_thr), axis=1)
            feature = f'X_{col_gn}_{gn_thr}'
            print(f'{col_gn}_{gn_thr} subsets...')
            gen_list = ['CDS', 'start_codon', 'stop_codon']
            df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
            gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
            df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
            gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
            #gen_list = ['.']
            df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)

    print('frequence calc...')
    cols = ['AD2_SAMPLES', 'AD3_SAMPLES', 'AD4_SAMPLES', 'AD5_SAMPLES', 
            'SQ5_SAMPLES', 'SQ10_SAMPLES', 'SQ20_SAMPLES', 'SQ30_SAMPLES', 'SQ40_SAMPLES', 'SQ50_SAMPLES', 'SQ60_SAMPLES', 'SQ70_SAMPLES']
    for col in cols:
        print(f'   {col}')
        df[f'freq_{col}'] = df.apply(lambda row: get_freq(row, col, parents_set_lr, parents_set_sr), axis=1)

    print('count the number of samples in filter columns...')
    cols = ['ZERO_COV_SAMPLES', 'SQ5_SAMPLES', 'SQ10_SAMPLES', 'SQ20_SAMPLES', 'SQ30_SAMPLES', 
            'SQ40_SAMPLES', 'SQ50_SAMPLES', 'SQ60_SAMPLES', 'SQ70_SAMPLES', 
            'AD2_SAMPLES', 'AD3_SAMPLES', 'AD4_SAMPLES', 'AD5_SAMPLES', 'HET_SAMPLES', 'HOMALT_SAMPLES', 'IL_SAMPLES', 'IL_GT_SAMPLES', 'SV2_GT_SAMPLES']
    for col in cols:
        print(col)
        df[f'NUM_{col}'] = df.apply(lambda row: count_num(row, col), axis=1)

    print('count number of case/control...')
    cols = ['SQ5_SAMPLES', 'SQ10_SAMPLES', 'SQ20_SAMPLES', 'SQ30_SAMPLES', 'SQ40_SAMPLES', 'SQ50_SAMPLES', 'SQ60_SAMPLES', 'SQ70_SAMPLES', 
            'AD2_SAMPLES', 'AD3_SAMPLES', 'AD4_SAMPLES', 'AD5_SAMPLES', 'HET_SAMPLES', 'HOMALT_SAMPLES', 'IL_SAMPLES', 'IL_GT_SAMPLES', 'SV2_GT_SAMPLES']
    df[[f'{col}_CASE_NUM' for col in cols]+[f'{col}_CONTROL_NUM' for col in cols]] = df.apply(lambda row: status_num(row, cols), 
                                                                                                         axis=1, result_type='expand')


    print('IL samples names to LR sample names...')
    rename_IL_to_LR = {'REACH000236_IL': 'REACH000236_PB', 
                       'REACH000436_IL': 'REACH000436_PB', 
                       'REACH000530_IL': 'REACH000530_PB', 
                       'REACH000531_IL': 'REACH000531_ONT', 
                       'REACH000532_IL': 'REACH000532_ONT'}
    df['IL_SAMPLES_LR'] = df.apply(lambda row: ','.join([x if x not in rename_IL_to_LR else rename_IL_to_LR[x] 
                                                         for x in row['IL_SAMPLES'].split(',')]), axis=1)
    df['HET_SAMPLES_LR'] = df.apply(lambda row: ','.join([x if x not in rename_IL_to_LR else rename_IL_to_LR[x] 
                                                         for x in row['HET_SAMPLES'].split(',')]), axis=1)
    df['HOMALT_SAMPLES_LR'] = df.apply(lambda row: ','.join([x if x not in rename_IL_to_LR else rename_IL_to_LR[x] 
                                                         for x in row['HOMALT_SAMPLES'].split(',')]), axis=1)
    df['IL_GT_SAMPLES_LR'] = df.apply(lambda row: ','.join([x if x not in rename_IL_to_LR else rename_IL_to_LR[x] 
                                                         for x in row['IL_GT_SAMPLES'].split(',')]), axis=1)
    cols = ['AD2_SAMPLES', 'AD3_SAMPLES', 'AD4_SAMPLES', 'AD5_SAMPLES', 
            'SQ5_SAMPLES', 'SQ10_SAMPLES', 'SQ20_SAMPLES', 'SQ30_SAMPLES', 'SQ40_SAMPLES', 'SQ50_SAMPLES', 'SQ60_SAMPLES', 'SQ70_SAMPLES']
    for col in cols:
        df[col] = df.apply(lambda row: ','.join([x if x not in rename_IL_to_LR else rename_IL_to_LR[x] 
                                                 for x in row[col].split(',')]), axis=1)
    
    print('generate alternative PLATFOMR from cross validation with IL calls...')
    col_plat_old = 'PLATFORM'
    col_lr = 'AD2_SAMPLES'
    df['PLATFORM_CV'] =  df.apply(lambda row: platform_redef(row, col_plat_old, col_lr), axis=1)

    print('count number of transmissions...')
    cols = ['AD2_SAMPLES', 'AD3_SAMPLES', 'AD4_SAMPLES', 'AD5_SAMPLES', 
            'SQ5_SAMPLES', 'SQ10_SAMPLES', 'SQ20_SAMPLES', 'SQ30_SAMPLES', 'SQ40_SAMPLES', 'SQ50_SAMPLES', 'SQ60_SAMPLES', 'SQ70_SAMPLES']
    for col in cols:
        print(f'   {col}')
        df[[f'NUM_tran_case_{col}', f'NUM_tran_control_{col}', 
        f'NUM_nontran_case_{col}', f'NUM_nontran_control_{col}']] = df.apply(lambda row: get_trans(row, col, mom_kids_dict, dad_kids_dict, 
                                                                                                   parent_kids_dict, mate_dict, aff_dict), 
                                                                            axis=1, result_type='expand')
        #df[[f'NUM_tran_case_asd_{col}', f'NUM_tran_control_asd_{col}', 
        #f'NUM_nontran_case_asd_{col}', f'NUM_nontran_control_asd_{col}']] = df.apply(lambda row: get_trans(row, col, mom_kids_dict, dad_kids_dict, 
        #                                                                                           parent_kids_dict, mate_dict, aff_dict_asd), 
        #                                                                    axis=1, result_type='expand')
    
    biotypes = ['lncRNA' ,'miRNA' ,'misc_RNA' ,'processed_transcript' ,'snoRNA' ,'snRNA' ,'TEC', 'protein_coding']
    for biotype in biotypes:
        print(f'biotype: {biotype}')
        df[f'X_{biotype}'] = df.apply(lambda row: xfeature(row, biotype), axis=1)

    ####################################
    print('protein coding subsets...')
    feature = 'X_protein_coding'
    gen_list = ['CDS', 'start_codon', 'stop_codon']
    df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
    df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
    #gen_list = ['.']
    df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)

    print('pli...')
    pli_thr = .99
    df['X_PLI'] = df.apply(lambda row: xpli(row, pli_thr), axis=1)

    print('pli >0.9...')
    pli_thr = .9
    df['X_PLIp9'] = df.apply(lambda row: xpli(row, pli_thr), axis=1)

    print('low pli...')
    pli_thr = .9
    df['X_LOWPLI'] = df.apply(lambda row: xlowpli(row, pli_thr), axis=1)

    print('loeuf...')
    l_thr = .37
    #l_thr = .1
    df['X_LOEUF'] = df.apply(lambda row: xloeuf(row, l_thr), axis=1)

    print('ASD...')
    f_thr = .05
    f_type = 'ASD'
    df['X_FDR_ASD'] = df.apply(lambda row: xfdr(row, f_type, f_thr), axis=1)

    print('DD...')
    f_type = 'DD'
    df['X_FDR_DD'] = df.apply(lambda row: xfdr(row, f_type, f_thr), axis=1)

    print('NDD...')
    f_type = 'NDD'
    df['X_FDR_NDD'] = df.apply(lambda row: xfdr(row, f_type, f_thr), axis=1)

    print('intergenic...')
    df['X_intergenic'] = df.apply(lambda row: xintergenic(row), axis=1)

    print('genic...')
    df['X_genic'] = df.apply(lambda row: xgenic(row), axis=1)

    ####################################
    print('genic subsets...')
    feature = 'X_genic'
    gen_list = ['CDS', 'start_codon', 'stop_codon']
    df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
    df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
    #gen_list = ['.']
    df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    ####################################
    print('pli subsets...')
    feature = 'X_PLI'
    gen_list = ['CDS', 'start_codon', 'stop_codon']
    df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
    df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
    #gen_list = ['.']
    df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)

    ####################################
    print('pli >0.9 subsets...')
    feature = 'X_PLIp9'
    gen_list = ['CDS', 'start_codon', 'stop_codon']
    df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
    df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
    #gen_list = ['.']
    df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)

    ####################################
    print('low pli subsets...')
    feature = 'X_LOWPLI'
    gen_list = ['CDS', 'start_codon', 'stop_codon']
    df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
    df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
    #gen_list = ['.']
    df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    ####################################
    print('loeuf subsets...')
    feature = 'X_LOEUF'
    gen_list = ['CDS', 'start_codon', 'stop_codon']
    df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
    df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
    #gen_list = ['.']
    df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)

    ####################################
    print('ASD subsets...')
    feature = 'X_FDR_ASD'
    gen_list = ['CDS', 'start_codon', 'stop_codon']
    df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
    df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
    #gen_list = ['.']
    df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)

    ####################################
    print('DD subsets...')
    feature = 'X_FDR_DD'
    gen_list = ['CDS', 'start_codon', 'stop_codon']
    df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
    df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
    #gen_list = ['.']
    df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)

    ####################################
    print('NDD subsets...')
    feature = 'X_FDR_NDD'
    gen_list = ['CDS', 'start_codon', 'stop_codon']
    df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
    df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
    gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
    #gen_list = ['.']
    df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)

    print('LoF...')
    df['LOF'] = df.apply(lambda row: lof(row), axis=1)

    print('LoF constrained subsets...')
    feature1 = 'LOF'
    feature2 = 'X_PLI_cds'
    df['X_LOF_PLI_cds'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_PLIp9_cds'
    df['X_LOF_PLIp9_cds'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_LOWPLI_cds'
    df['X_LOF_LOWPLI_cds'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_LOEUF_cds'
    df['X_LOF_LOEUF_cds'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_FDR_ASD_cds'
    df['X_LOF_FDR_ASD_cds'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_FDR_DD_cds'
    df['X_LOF_FDR_DD_cds'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_FDR_NDD_cds'
    df['X_LOF_FDR_NDD_cds'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    
    print('FB PR...')
    fb_set = set(['1_TssA', '2_TssAFlnk', '10_TssBiv', '11_BivFlnk'])
    df['X_FB_PR'] = df.apply(lambda row: xfb_pr_enh(row, fb_set), axis=1)

    print('FB ENH...')
    fb_set = set(['6_EnhG', '7_Enh', '12_EnhBiv'])
    df['X_FB_ENH'] = df.apply(lambda row: xfb_pr_enh(row, fb_set), axis=1)

    print('FANTOM...')
    df['X_FANTOM'] = df.apply(lambda row: xfantom(row), axis=1)

    ### placeholder for gnocchi calc
    

    print('protein_coding CDS subsets...')
    feature1 = 'X_protein_coding_cds'
    feature2 = 'X_PLI'
    df['X_protein_coding_cds_pli'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_LOEUF'
    df['X_protein_coding_cds_loeuf'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_FDR_ASD'
    df['X_protein_coding_cds_fdr_asd'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_FDR_DD'
    df['X_protein_coding_cds_fdr_dd'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)
    feature2 = 'X_FDR_NDD'
    df['X_protein_coding_cds_fdr_ndd'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)

    print('brain span genes...')
    df['X_DEV_BR_GENE_10'] = df.apply(lambda row: xdev_br(row, br_flt_10), axis=1)
    df['X_DEV_BR_GENE_20'] = df.apply(lambda row: xdev_br(row, br_flt_20), axis=1)
    df['X_DEV_BR_GENE_30'] = df.apply(lambda row: xdev_br(row, br_flt_30), axis=1)
    df['X_DEV_BR_GENE_40'] = df.apply(lambda row: xdev_br(row, br_flt_40), axis=1)
    df['X_DEV_BR_GENE_50'] = df.apply(lambda row: xdev_br(row, br_flt_50), axis=1)

    ####################################
    print('dev brain genes subsets...')
    for rpkm in [10, 20, 30, 40, 50]:
        feature = f'X_DEV_BR_GENE_{rpkm}'
        gen_list = ['CDS', 'start_codon', 'stop_codon']
        df[f'{feature}_cds'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
        gen_list = ['five_prime_UTR', 'three_prime_UTR', 'TSS']
        df[f'{feature}_utr'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)
    
        gen_list = ['CTCF-only' ,'dELS' ,'DNase-H3K4me3' ,'exon' ,'pELS' ,'PLS', '.']
        #gen_list = ['.']
        df[f'{feature}_intron'] = df.apply(lambda row: xgencode(row, gen_list, feature), axis=1)

    print('LoF dev brain...')
    feature1 = 'LOF'
    for rpkm in [10, 20, 30, 40, 50]:
        feature2 = f'X_DEV_BR_GENE_{rpkm}_cds'
        df[f'X_LOF_DEV_BR_{rpkm}_cds'] = df.apply(lambda row: x2features(row, feature1, feature2), axis=1)

    print('dev brain and PLI...')
    feature1 = 'X_PLIp9'
    for feature2 in ['LOF_DEV_BR_10_cds', 
                     'DEV_BR_GENE_10', 'DEV_BR_GENE_10_cds', 'DEV_BR_GENE_10_utr', 'DEV_BR_GENE_10_intron']:
        df[f'{feature1}_{feature2}'] = df.apply(lambda row: x2features(row, feature1, f'X_{feature2}'), axis=1)
    
    print('writing the output...')
    df.to_csv(f'{dir_out}/test.tsv', sep='\t', header=True, index=False)
    print('+'*40)
    
    return df

In [6]:
file_cov = '../data/coverages_mod.tsv'
df_cov = pd.read_table(file_cov, header=0, sep='\t')
multiplat_samples = ['REACH000236', 'REACH000530', 'REACH000531', 'REACH000532', 'REACH000436']
df_cov['SAMPLE'] = df_cov.apply(lambda row: row['SAMPLE']+'_'+row['COHORT'] if row['SAMPLE'] in multiplat_samples else row['SAMPLE'], axis=1)
display(df_cov)

def get_plat(sample):
    try:
        ret = df_cov[df_cov.SAMPLE == sample]['COHORT'].values[0]
    except:
        print(f'problem with samples: {sample}')
        ret = ''
    return ret

plat_dict = {sample: get_plat(sample) for sample in df_cov.SAMPLE.tolist()}

,SAMPLE,COHORT,MEAN_COVERAGE
0,HG004,ONT,81.44
1,REACH000626,ONT,10.51
2,REACH000546,ONT,10.87
3,REACH000683,ONT,9.97
4,REACH000563,ONT,8.68
...,...,...,...
283,REACH000284,PB,3.43
284,REACH000672,PB,5.02
285,REACH000292,PB,9.00
286,REACH000066,PB,1.61


In [7]:
file_brainspan = '../data/gene_list_RPKM_10.csv'
dtype = {'gene_id': str, 'entrez_id': str}
df_br = pd.read_table(file_brainspan, sep=',', header=0, dtype=dtype)
br_flt_10 = df_br.loc[df_br.mean_rpkm>10].gene_symbol.tolist()
br_flt_20 = df_br.loc[df_br.mean_rpkm>20].gene_symbol.tolist()
br_flt_30 = df_br.loc[df_br.mean_rpkm>30].gene_symbol.tolist()
br_flt_40 = df_br.loc[df_br.mean_rpkm>40].gene_symbol.tolist()
br_flt_50 = df_br.loc[df_br.mean_rpkm>50].gene_symbol.tolist()
print(df_br.shape)
print(len(br_flt_10))
print(len(br_flt_20))
print(len(br_flt_30))
print(len(br_flt_40))
print(len(br_flt_50))

(5824, 32)
5824
3045
1983
1392
1078


In [8]:
file_psam = '../data/REACH_LR_platform.psam'
df_p = pd.read_table(file_psam, sep='\t', header=None, names = ['famid', 'sample_id', 'dad', 'mom', 'sex', 'phen'])
display(df_p)

# make mom and dad dictionary
sample_dad_dict = {}
sample_mom_dict = {}
for sample, dad, mom in zip(df_p.sample_id, df_p.dad, df_p.mom):
    #print(sample, dad, mom)
    sample_dad_dict[sample] = dad
    sample_mom_dict[sample] = mom

print('complete trios:')
df_trios = df_p.loc[df_p.dad.isin(df_p.sample_id) & df_p.mom.isin(df_p.sample_id)]
display(df_trios)
print(df_trios.sex.value_counts())
print(df_trios.phen.value_counts())

parents_set_lr = set(df_trios.dad.tolist()) | set(df_trios.mom.tolist())
print(f'len(parents_set_lr): {len(parents_set_lr)}')
#print(parents_set_lr)

rename_samples_dict_IL = {'REACH000236_PB': 'REACH000236_IL', 
                          'REACH000436_PB': 'REACH000436_IL', 
                          'REACH000530_PB': 'REACH000530_IL', 
                          'REACH000531_ONT': 'REACH000531_IL', 
                          'REACH000532_ONT': 'REACH000532_IL'}
parents_set_sr = {x if x not in rename_samples_dict_IL else rename_samples_dict_IL[x] for x in parents_set_lr}
print(f'len(parents_set_sr): {len(parents_set_sr)}')
#print(parents_set_sr)

parent_kids_dict = {}
mom_kids_dict = {}
dad_kids_dict = {}
for parent in parents_set_lr:
    parent_kids_dict[parent] = df_trios.loc[(df_trios.dad == parent) | (df_trios.mom == parent)].sample_id.tolist()
    if parent in df_trios.dad.tolist():
        dad_kids_dict[parent] = df_trios.loc[(df_trios.dad == parent)].sample_id.tolist()
    if parent in df_trios.mom.tolist():
        mom_kids_dict[parent] = df_trios.loc[(df_trios.mom == parent)].sample_id.tolist()
print(f'len(parent_kids_dict): {len(parent_kids_dict)}')
#print(parent_kids_dict)
print(f'len(mom_kids_dict): {len(mom_kids_dict)}')
#print(mom_kids_dict)
print(f'len(dad_kids_dict): {len(dad_kids_dict)}')
#print(dad_kids_dict)

mate_dict = {}
for dad, mom in zip(df_trios.dad, df_trios.mom):
    mate_dict[dad] = mom
    mate_dict[mom] = dad
print(f'len(mate_dict): {len(mate_dict)}')

,famid,sample_id,dad,mom,sex,phen
0,2323,2323-2-1,0,0,2,2
1,2602,2602-2-1,0,0,2,2
2,3392,3392-2-3,0,0,2,2
3,3538,3538-2-1,0,0,2,2
4,3939,3939-3-1,0,0,2,2
...,...,...,...,...,...,...
275,F0270,REACH000681,REACH000683,REACH000682,1,2
276,F0270,REACH000682,0,0,2,2
277,F0270,REACH000683,0,0,1,2
278,F0270,REACH000684,REACH000683,REACH000682,1,2


complete trios:


,famid,sample_id,dad,mom,sex,phen
10,F0026,REACH000026,REACH000270,REACH000269,1,2
11,F0058,REACH000058,REACH000440,REACH000439,1,2
12,F0065,REACH000065,REACH000067,REACH000066,1,2
15,F0078,REACH000086,REACH000088,REACH000087,1,2
18,F0078,REACH000089,REACH000088,REACH000087,2,2
...,...,...,...,...,...,...
271,F0266,REACH000660,REACH000662,REACH000661,2,2
274,F0266,REACH000663,REACH000662,REACH000661,1,1
275,F0270,REACH000681,REACH000683,REACH000682,1,2
278,F0270,REACH000684,REACH000683,REACH000682,1,2


sex
1    74
2    43
Name: count, dtype: int64
phen
2    92
1    25
Name: count, dtype: int64
len(parents_set_lr): 126
len(parents_set_sr): 126
len(parent_kids_dict): 126
len(mom_kids_dict): 63
len(dad_kids_dict): 63
len(mate_dict): 126


In [9]:
meta_file = '../data/REACH_sample_info.tsv'
df_meta = pd.read_table(meta_file, sep='\t', header=0)

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
df_meta['Sample_ID'] = df_meta.apply(lambda row: rename_samples_dict[row['Sample_ID']] 
                                     if row['Sample_ID'] in rename_samples_dict else row['Sample_ID'], axis=1)

aff_dict = {}
for sample, aff in zip(df_meta['Sample_ID'].tolist(), df_meta['Affected'].tolist()):
    aff_dict[sample] = aff
    
aff_dict_asd = {}
for sample, aff in zip(df_meta['Sample_ID'].tolist(), df_meta['Affected_ASD'].tolist()):
    aff_dict_asd[sample] = aff

In [10]:
### previously discovered variants
file_fdr = '../data/fdr_pvals_tada.tsv'
df_fdr = pd.read_table(file_fdr, sep='\t', header=0, index_col=None)
print('df_fdr:')
display(df_fdr)
asd_genes = df_fdr.loc[df_fdr.FDR_TADA_ASD <= 0.05].gene_gencodeV33.tolist()
dd_genes = df_fdr.loc[df_fdr.FDR_TADA_DD <= 0.05].gene_gencodeV33.tolist()
ndd_genes = df_fdr.loc[df_fdr.FDR_TADA_NDD <= 0.05].gene_gencodeV33.tolist()
print(f'asd: {len(asd_genes)}')
#print(f'asd: {(asd_genes)}')
print(f'dd: {len(dd_genes)}')
print(f'ndd: {len(ndd_genes)}')

file_pli = '../data/gene_max_pli.tsv'
df_pli = pd.read_table(file_pli, sep='\t', header=0, index_col=None).set_index('gene')
print('df_pli:')
display(df_pli)

file_lf = '../data/gene_min_loeuf.tsv'
df_lf = pd.read_table(file_lf, sep='\t', header=0, index_col=None).set_index('gene')
print('df_lf:')
display(df_lf)

print('dnLOF and dnMIS SNV/indels')
file_dn_snv = '../data/Antaki2022_supplementary_tables_ST5.tsv'
df_dn_snv = pd.read_table(file_dn_snv, sep='\t', header=0, index_col=None, keep_default_na=False)
df_dn_snv['PLI'] = df_dn_snv.gene.map(df_pli['lof.pLI'])
df_dn_snv['LOEUF'] = df_dn_snv.gene.map(df_lf['lof.oe_ci.upper'])
df_dn_snv['FDR_ASD'] = df_dn_snv.gene.map(lambda x: 1 if x in asd_genes else 0)
df_dn_snv['FDR_DD'] = df_dn_snv.gene.map(lambda x: 1 if x in dd_genes else 0)
df_dn_snv['FDR_NDD'] = df_dn_snv.gene.map(lambda x: 1 if x in ndd_genes else 0)
df_dn_snv['DEV_BR_GENE_10'] = df_dn_snv.gene.map(lambda x: 1 if x in br_flt_10 else 0)
df_dn_snv = df_dn_snv.loc[(df_dn_snv.iid.str.contains('REACH'))]
display(df_dn_snv.shape)
print(f" fdr asd: {df_dn_snv['FDR_ASD'].sum()}")
print(f" fdr dd: {df_dn_snv['FDR_DD'].sum()}")
print(f" fdr ndd: {df_dn_snv['FDR_NDD'].sum()}")
print(f" fdr brain genes: {df_dn_snv['DEV_BR_GENE_10'].sum()}")
print('df_dn_snv:')
display(df_dn_snv)
this_df_dn = df_dn_snv[['loss_of_function', 'missense', 'PLI', 'LOEUF', 'FDR_ASD', 'FDR_DD', 'FDR_NDD', 'DEV_BR_GENE_10']].copy()
this_df_dn['samples_all'] = df_dn_snv['iid']
# filter constraint genes
this_df_dn = this_df_dn.loc[((this_df_dn.PLI>=0.9) | (this_df_dn.LOEUF<=0.37) | 
                             (this_df_dn.FDR_ASD==1) | (this_df_dn.FDR_DD==1) | (this_df_dn.FDR_NDD==1))]
print('this_df_dn:')
display(this_df_dn.shape)
display(this_df_dn)

print('inhLOF SNV/indels')
file_inh_snv = '../data/Antaki2022_supplementary_tables_ST8.tsv'
df_inh_snv = pd.read_table(file_inh_snv, sep='\t', header=0, index_col=None, keep_default_na=False)
df_inh_snv['PLI'] = df_inh_snv.gene.map(df_pli['lof.pLI'])
df_inh_snv['LOEUF'] = df_inh_snv.gene.map(df_lf['lof.oe_ci.upper'])
df_inh_snv['FDR_ASD'] = df_inh_snv.gene.map(lambda x: 1 if x in asd_genes else 0)
df_inh_snv['FDR_DD'] = df_inh_snv.gene.map(lambda x: 1 if x in dd_genes else 0)
df_inh_snv['FDR_NDD'] = df_inh_snv.gene.map(lambda x: 1 if x in ndd_genes else 0)
df_inh_snv['DEV_BR_GENE_10'] = df_inh_snv.gene.map(lambda x: 1 if x in br_flt_10 else 0)
df_inh_snv = df_inh_snv.loc[(df_inh_snv.cases.str.contains('REACH')) | ((df_inh_snv.controls.str.contains('REACH')))]
display(df_inh_snv.shape)
print(f" fdr asd: {df_inh_snv['FDR_ASD'].sum()}")
print(f" fdr dd: {df_inh_snv['FDR_DD'].sum()}")
print(f" fdr ndd: {df_inh_snv['FDR_NDD'].sum()}")
print(f" fdr brain genes: {df_inh_snv['DEV_BR_GENE_10'].sum()}")
print('df_inh_snv:')
display(df_inh_snv)
this_df_inh = df_inh_snv[['PLI', 'LOEUF', 'FDR_ASD', 'FDR_DD', 'FDR_NDD', 'DEV_BR_GENE_10']].copy()
this_df_inh['samples_all'] = df_inh_snv.apply(lambda row: ','.join([x for x in [row.cases, row.controls, 
                                                                         row.fathers, row.mothers] if x != '']), axis=1)
# filter constraint genes
this_df_inh = this_df_inh.loc[((this_df_inh.PLI>=0.9) | (this_df_inh.LOEUF<=0.37) | 
                             (this_df_inh.FDR_ASD==1) | (this_df_inh.FDR_DD==1) | (this_df_inh.FDR_NDD==1))]
print('this_df_inh:')
display(this_df_inh.shape)
display(this_df_inh)

print('Merged denovo and inheritted one row per sample')
this_df_dn_inh = pd.concat([this_df_dn, this_df_inh], axis=0, ignore_index=True)
this_df_dn_inh['samples_all'] = this_df_dn_inh.samples_all.str.split(',')
this_df_dn_inh = this_df_dn_inh.explode('samples_all')

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
this_df_dn_inh['samples_all'] = this_df_dn_inh.apply(lambda row: rename_samples_dict[row['samples_all']] 
                                             if row['samples_all'] in rename_samples_dict else row['samples_all'], axis=1)

print('this_df_dn_inh:')
display(this_df_dn_inh)

df_fdr:


,gene_gencodeV33,FDR_TADA_ASD,FDR_TADA_DD,FDR_TADA_NDD,p_TADA_ASD,p_TADA_DD,p_TADA_NDD
0,SYNGAP1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,CHD8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,ADNP,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,SHANK3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,SCN2A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
18123,PSG2,0.911036,0.913971,0.879392,0.999770,0.999656,0.999762
18124,NaN,0.911038,0.913973,0.879395,0.999827,0.999713,0.999821
18125,GJB2,0.911040,0.913975,0.879399,0.999885,0.999771,0.999881
18126,PSG1,0.911042,0.913979,0.879403,0.999942,0.999885,0.999940


asd: 185
dd: 477
ndd: 664
df_pli:


,lof.pLI
gene,
A1BG,1.814400e-07
A1CF,6.691700e-03
A2M,7.412300e-02
A2ML1,1.996200e-01
A3GALT2,1.007500e-06
...,...
ZYG11A,3.084800e-08
ZYG11B,8.055900e-01
ZYX,3.807700e-02


df_lf:


...

this_df_inh:


(125, 7)

,PLI,LOEUF,FDR_ASD,FDR_DD,FDR_NDD,DEV_BR_GENE_10,samples_all
275,1.000000,0.126,0,0,0,1,"REACH000544,REACH000547,REACH000546"
281,1.000000,0.295,0,0,0,1,"REACH000760,REACH000759"
297,1.000000,0.131,0,1,1,1,"REACH000650,REACH000652"
589,1.000000,0.332,0,0,0,0,"REACH000372,REACH000430"
688,1.000000,0.151,1,1,1,1,"REACH000668,REACH000670"
...,...,...,...,...,...,...,...
10597,0.000007,0.598,0,1,1,0,"REACH000668,REACH000669"
10615,1.000000,0.215,1,0,1,0,"REACH000166,REACH000168,REACH000164"
10745,0.936450,0.342,0,0,0,1,"REACH000586,REACH000587"
10774,0.988370,0.291,0,0,0,0,"REACH000387,REACH000388"


Merged denovo and inheritted one row per sample
this_df_dn_inh:


,loss_of_function,missense,PLI,LOEUF,FDR_ASD,FDR_DD,FDR_NDD,DEV_BR_GENE_10,samples_all
0,0.0,1.0,0.99979,0.415,0,0,0,1,REACH000089
1,1.0,0.0,1.00000,0.271,0,0,0,0,REACH000718
2,0.0,1.0,1.00000,0.183,0,0,1,1,REACH000741
3,0.0,1.0,1.00000,0.220,0,0,1,1,REACH000774
4,1.0,0.0,1.00000,0.366,0,0,0,1,REACH000097
...,...,...,...,...,...,...,...,...,...
150,NaN,NaN,0.93645,0.342,0,0,0,1,REACH000587
151,NaN,NaN,0.98837,0.291,0,0,0,0,REACH000387
151,NaN,NaN,0.98837,0.291,0,0,0,0,REACH000388
152,NaN,NaN,0.99978,0.124,0,0,0,0,REACH000252


In [11]:
file_pca = '../data/master_phen_4.SBayesR.20210915.csv'

df_pca = pd.read_table(file_pca, sep=',', header=0)
cols = ['fid', 'iid', 'phen','sex','cohort','duo','family','is_eur','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10', 
        'PC1_PAT','PC2_PAT','PC3_PAT','PC4_PAT','PC5_PAT','PC6_PAT','PC7_PAT','PC8_PAT','PC9_PAT','PC10_PAT', 
        'PC1_MAT','PC2_MAT','PC3_MAT','PC4_MAT','PC5_MAT','PC6_MAT','PC7_MAT','PC8_MAT','PC9_MAT','PC10_MAT']
df_pca = df_pca[cols]

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
df_pca['iid'] = df_pca.apply(lambda row: rename_samples_dict[row['iid']] if row['iid'] in rename_samples_dict else row['iid'], axis=1)
#display(df_pca)

# first filter for the kids, then add parents to them
df_pca_flt = df_pca.loc[df_pca.iid.isin(df_trios.sample_id)].copy()
df_pca_flt['rel'] = 'C'
print('df_pca_flt:')
display(df_pca_flt)

df_parents = pd.DataFrame()
for index, row in df_pca_flt.iterrows():
    sample = row['iid']
    fid = row['fid']
    phen = row['phen']
    is_eur = row['is_eur']
    pc_mat = [row[f'PC{x}_MAT'] for x in range(1,11)]
    pc_pat = [row[f'PC{x}_PAT'] for x in range(1,11)]
    dad = sample_dad_dict[sample]
    mom = sample_mom_dict[sample]
    
    dict_mat = {'fid': [fid], 'iid': [mom], 'phen': ['.'], 'sex': ['Female'], 'cohort': [1], 'duo': ['.'], 'is_eur': [is_eur]}
    dict_mat.update({f'PC{x}': [pc_mat[x-1]] for x in range(1,11)})
    df_mat = pd.DataFrame(dict_mat)

    dict_pat = {'fid': [fid], 'iid': [dad], 'phen': ['.'], 'sex': ['Male'], 'cohort': [1], 'duo': ['.'], 'is_eur': [is_eur]}
    dict_pat.update({f'PC{x}': [pc_pat[x-1]] for x in range(1,11)})
    df_pat = pd.DataFrame(dict_pat)
    
    if df_parents.shape[0] == 0 or mom not in df_parents.iid.tolist():
        df_parents = pd.concat([df_parents, df_mat])
    if df_parents.shape[0] == 0 or dad not in df_parents.iid.tolist():
        df_parents = pd.concat([df_parents, df_pat])

df_parents['rel'] = 'P'
print('df_parents:')
display(df_parents)

df_pca_flt = pd.concat([df_pca_flt, df_parents])
print('df_pca_flt:')
display(df_pca_flt)

df_pca_flt = pd.merge(df_pca_flt, df_meta[['Sample_ID', 'Affected', 'Affected_ASD']], how='inner', left_on='iid', right_on='Sample_ID')
df_pca_flt = pd.merge(df_pca_flt, df_cov[['SAMPLE', 'MEAN_COVERAGE']], how='inner', left_on='iid', right_on='SAMPLE')

df_pca_flt['Phenotype'] = df_pca_flt.apply(lambda row: 1 if row.Affected == 'Yes' else 0, axis=1)
df_pca_flt['Phenotype_ASD'] = df_pca_flt.apply(lambda row: 1 if row.Affected_ASD == 'Yes' else 0, axis=1)
df_pca_flt['Platform'] = df_pca_flt.apply(lambda row: get_plat(row.iid), axis=1)
#df_pca_flt.to_csv('investigate2.tsv', sep='\t', index=None)

df_pca_flt['SNV_CAR'] = df_pca_flt.apply(lambda row: 1 if row['iid'] in this_df_dn_inh.samples_all.tolist() else 0, axis=1)
print(f"# samples with SNV vars: {df_pca_flt['SNV_CAR'].sum()}")

df_pca_flt:


,fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,...,PC2_MAT,PC3_MAT,PC4_MAT,PC5_MAT,PC6_MAT,PC7_MAT,PC8_MAT,PC9_MAT,PC10_MAT,rel
136,F0001,REACH000001,ASD,Male,1,trio,1,1,0.008627,-0.005607,...,0.000410,0.002845,-0.019312,-0.013769,-0.014384,0.007249,0.002096,0.005399,-0.014983,C
137,F0026,REACH000026,ASD,Male,1,trio,1,0,0.095364,-0.045230,...,-0.047780,-0.017435,0.153686,0.005292,-0.009487,-0.020773,0.034190,0.015895,-0.011925,C
139,F0058,REACH000058,ASD,Male,1,trio,1,1,0.002422,-0.009216,...,-0.003289,0.003757,-0.004828,0.007650,0.005633,0.032103,0.009510,0.020007,0.016383,C
140,F0065,REACH000065,ASD,Male,1,trio,1,0,0.252774,0.107933,...,0.049799,0.158347,0.011411,0.004728,0.007690,-0.006605,0.008261,-0.012000,-0.009906,C
146,F0078,REACH000086,ASD,Male,1,trio,2,0,0.044785,0.038065,...,0.080387,0.132056,0.029583,0.020156,0.007971,-0.002383,0.014468,0.006253,-0.018303,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,F0290,REACH000762,ASD,Male,1,trio,1,1,-0.027901,0.000946,...,0.003052,-0.009075,-0.014136,-0.005499,-0.008555,0.015644,0.004869,0.011004,-0.019684,C
409,F0290,REACH000765,CON,Female,1,trio,1,1,-0.030809,-0.001867,...,0.003052,-0.009075,-0.014136,-0.005499,-0.008555,0.015644,0.004869,0.011004,-0.019684,C
410,F0290,REACH000766,CON,Female,1,trio,1,1,-0.028469,-0.005167,...,0.003052,-0.009075,-0.014136,-0.005499,-0.008555,0.015644,0.004869,0.011004,-0.019684,C
411,F0291,REACH000767,ASD,Male,1,trio,1,0,0.087690,-0.012273,...,-0.016917,-0.021358,0.151522,-0.035297,0.008383,0.005679,0.020179,-0.022316,0.002755,C


df_parents:


,fid,iid,phen,sex,cohort,duo,is_eur,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,rel
0,F0001,REACH000107,.,Female,1,.,1,-0.022403,0.000410,0.002845,-0.019312,-0.013769,-0.014384,0.007249,0.002096,0.005399,-0.014983,P
0,F0001,REACH000226,.,Male,1,.,1,0.024023,-0.013482,-0.003936,0.129597,0.030587,-0.007727,-0.002212,-0.022159,0.005150,-0.000824,P
0,F0026,REACH000269,.,Female,1,.,0,0.100551,-0.047780,-0.017435,0.153686,0.005292,-0.009487,-0.020773,0.034190,0.015895,-0.011925,P
0,F0026,REACH000270,.,Male,1,.,0,0.091671,-0.037847,-0.022134,0.163193,0.007864,-0.006841,-0.023716,0.040800,0.006666,-0.012916,P
0,F0058,REACH000439,.,Female,1,.,1,-0.025327,-0.003289,0.003757,-0.004828,0.007650,0.005633,0.032103,0.009510,0.020007,0.016383,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,F0288,REACH000756,.,Male,1,.,0,0.204041,0.127559,0.190286,0.013553,0.006706,-0.001266,-0.010150,0.006605,-0.006867,0.006560,P
0,F0290,REACH000763,.,Female,1,.,1,-0.030630,0.003052,-0.009075,-0.014136,-0.005499,-0.008555,0.015644,0.004869,0.011004,-0.019684,P
0,F0290,REACH000764,.,Male,1,.,1,-0.027327,-0.004878,-0.003881,0.000495,0.003727,-0.001647,-0.008831,0.016730,-0.005315,0.006958,P
0,F0291,REACH000768,.,Female,1,.,0,0.091442,-0.016917,-0.021358,0.151522,-0.035297,0.008383,0.005679,0.020179,-0.022316,0.002755,P


df_pca_flt:


,fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,...,PC2_MAT,PC3_MAT,PC4_MAT,PC5_MAT,PC6_MAT,PC7_MAT,PC8_MAT,PC9_MAT,PC10_MAT,rel
136,F0001,REACH000001,ASD,Male,1,trio,1.0,1,0.008627,-0.005607,...,0.000410,0.002845,-0.019312,-0.013769,-0.014384,0.007249,0.002096,0.005399,-0.014983,C
137,F0026,REACH000026,ASD,Male,1,trio,1.0,0,0.095364,-0.045230,...,-0.047780,-0.017435,0.153686,0.005292,-0.009487,-0.020773,0.034190,0.015895,-0.011925,C
139,F0058,REACH000058,ASD,Male,1,trio,1.0,1,0.002422,-0.009216,...,-0.003289,0.003757,-0.004828,0.007650,0.005633,0.032103,0.009510,0.020007,0.016383,C
140,F0065,REACH000065,ASD,Male,1,trio,1.0,0,0.252774,0.107933,...,0.049799,0.158347,0.011411,0.004728,0.007690,-0.006605,0.008261,-0.012000,-0.009906,C
146,F0078,REACH000086,ASD,Male,1,trio,2.0,0,0.044785,0.038065,...,0.080387,0.132056,0.029583,0.020156,0.007971,-0.002383,0.014468,0.006253,-0.018303,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,F0288,REACH000756,.,Male,1,.,NaN,0,0.204041,0.127559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
0,F0290,REACH000763,.,Female,1,.,NaN,1,-0.030630,0.003052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
0,F0290,REACH000764,.,Male,1,.,NaN,1,-0.027327,-0.004878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
0,F0291,REACH000768,.,Female,1,.,NaN,0,0.091442,-0.016917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P


# samples with SNV vars: 73


In [13]:
### # generate the big table
### if this file is not present in folder: make_raw_data, you should generate with code in that folder
### generates the output in folder: outputs
file_name = 'make_raw_data/combined_table.tsv'
df = get_big_table(file_name)
display(df)

/scratch/smmortazavi/job_40601570/ipykernel_3544310/1777979188.py:2: DtypeWarning: Columns (25,65,66,70,72,73,77,78,79,80,81,82,83,84,88,89,90,91,92,93,94,95,99,100,101,102,103,104,105,106,110,111,112,113,114,115,116,117,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table(file_name, sep='\t', header=0)


GNOCCHI_MAX_4...
GNOCCHI_MAX_4 subsets...
GNOCCHI_MAX_3...
GNOCCHI_MAX_3 subsets...
GNOCCHI_MAX_2...
GNOCCHI_MAX_2 subsets...
GNOCCHI_MAX_1...
GNOCCHI_MAX_1 subsets...
GNOCCHI_MEAN_4...
GNOCCHI_MEAN_4 subsets...
GNOCCHI_MEAN_3...
GNOCCHI_MEAN_3 subsets...
GNOCCHI_MEAN_2...
GNOCCHI_MEAN_2 subsets...
GNOCCHI_MEAN_1...
GNOCCHI_MEAN_1 subsets...
frequence calc...
   AD2_SAMPLES
   AD3_SAMPLES
   AD4_SAMPLES
   AD5_SAMPLES
   SQ5_SAMPLES
   SQ10_SAMPLES
   SQ20_SAMPLES
   SQ30_SAMPLES
   SQ40_SAMPLES
   SQ50_SAMPLES
   SQ60_SAMPLES
   SQ70_SAMPLES
count the number of samples in filter columns...
ZERO_COV_SAMPLES
SQ5_SAMPLES
SQ10_SAMPLES
SQ20_SAMPLES
SQ30_SAMPLES
SQ40_SAMPLES
SQ50_SAMPLES
SQ60_SAMPLES
SQ70_SAMPLES
AD2_SAMPLES
AD3_SAMPLES
AD4_SAMPLES
AD5_SAMPLES
HET_SAMPLES
HOMALT_SAMPLES
IL_SAMPLES
IL_GT_SAMPLES
SV2_GT_SAMPLES
count number of case/control...
IL samples names to LR sample names...
generate alternative PLATFOMR from cross validation with IL calls...
count number of transmissio

,CHROM,POS,END,ID,SVTYPE,PLATFORM,SVLEN,SRC,GENCODE,denovo_LR,...,X_LOF_DEV_BR_10_cds,X_LOF_DEV_BR_20_cds,X_LOF_DEV_BR_30_cds,X_LOF_DEV_BR_40_cds,X_LOF_DEV_BR_50_cds,X_PLIp9_LOF_DEV_BR_10_cds,X_PLIp9_DEV_BR_GENE_10,X_PLIp9_DEV_BR_GENE_10_cds,X_PLIp9_DEV_BR_GENE_10_utr,X_PLIp9_DEV_BR_GENE_10_intron
0,chr1,10000,180998,14346589,DUP,LR,170998,Lumpy,start_codon,.,...,0,0,0,0,0,0,0,0,0,0
1,chr1,10468,180108,Sniffles2.DEL.1DM0,DEL,LR,-169640,Sniffles_Lumpy,start_codon,.,...,0,0,0,0,0,0,0,0,0,0
2,chr1,14848,182483,14346592,DUP,LR,167635,Lumpy,start_codon,.,...,0,0,0,0,0,0,0,0,0,0
3,chr1,32024,32024,Sniffles2.INS.1M0,INS,LR,64,Sniffles,.,.,...,0,0,0,0,0,0,0,0,0,0
4,chr1,35141,35141,948107_1,BND,LR,24860,Lumpy,exon,.,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344741,chrY,56887898,56887898,6390796_2,BND,LR,.,Lumpy,.,.,...,0,0,0,0,0,0,0,0,0,0
344742,chrY,56887898,56887898,6390795_2,BND,LR,.,Lumpy,.,.,...,0,0,0,0,0,0,0,0,0,0
344743,chrY,56887898,56887898,10218686_2,BND,LR,.,Lumpy,.,.,...,0,0,0,0,0,0,0,0,0,0
344744,chrY,56887902,56887902,13088295_2,BND,LR,.,Lumpy,.,.,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def get_df_flt(df, flt_name, svtypes, ext_col=None, flt_sd_art=True, freq_col=None, freq_thr=None):
    df_flt = df.loc[((df[f'NUM_{flt_name}'] > 0) | (df['NUM_IL_SAMPLES'] > 0) | (df['NUM_IL_GT_SAMPLES'] > 0) | (df['NUM_SV2_GT_SAMPLES'] > 0)) & (df.SVTYPE.isin(svtypes))]
    #| (df['NUM_SV2_GT_SAMPLES'] > 0)
    if flt_sd_art:
        df_flt = df_flt.loc[~((df_flt.SD_ART==1) & (df_flt.PLATFORM=='LR'))]
    if freq_thr != None:
        df_flt = df_flt.loc[df_flt[f'freq_{freq_col}']<=freq_thr]
    return df_flt

In [15]:
biotypes = ['lncRNA' ,'miRNA' ,'misc_RNA' ,'processed_transcript' ,'snoRNA' ,'snRNA' ,'TEC', 'protein_coding']
pc_subsec = ['protein_coding_cds', 'protein_coding_utr', 'protein_coding_intron', 
             'protein_coding_cds_pli', 'protein_coding_cds_loeuf', 
             'protein_coding_cds_fdr_asd', 'protein_coding_cds_fdr_dd', 'protein_coding_cds_fdr_ndd']
consts_1 = ['PLI', 'LOEUF', 'FDR_ASD', 'FDR_DD', 'FDR_NDD', 'intergenic', 'genic']
consts_2 = ['FB_PR', 'FB_ENH', 'FANTOM']
consts_3 = ['GNOCCHI_MAX_1', 'GNOCCHI_MAX_2', 'GNOCCHI_MAX_3', 'GNOCCHI_MAX_4', 
            'GNOCCHI_MEAN_1', 'GNOCCHI_MEAN_2', 'GNOCCHI_MEAN_3', 'GNOCCHI_MEAN_4']
consts_3_subsec = ['GNOCCHI_MAX_4_cds', 'GNOCCHI_MAX_4_utr', 'GNOCCHI_MAX_4_intron', 
                   'GNOCCHI_MAX_3_cds', 'GNOCCHI_MAX_3_utr', 'GNOCCHI_MAX_3_intron', 
                   'GNOCCHI_MAX_2_cds', 'GNOCCHI_MAX_2_utr', 'GNOCCHI_MAX_2_intron']
consts_4 = ['EV_CONS_EL_BP_1', 'EV_CONS_EL_BP_50', 'EV_CONS_EL_BP_100', 
            'EV_CONS_EL_FRAC_2', 'EV_CONS_EL_FRAC_5', 'EV_CONS_EL_FRAC_10', 'S_HET_1', 'S_HET_10', 'S_HET_100']
consts_4_subsec1 = ['EV_CONS_EL_BP_1_cds', 'EV_CONS_EL_BP_1_utr', 'EV_CONS_EL_BP_1_ncd', 
                   'EV_CONS_EL_BP_50_cds', 'EV_CONS_EL_BP_50_utr', 'EV_CONS_EL_BP_50_ncd', 
                   'EV_CONS_EL_BP_100_cds', 'EV_CONS_EL_BP_100_utr', 'EV_CONS_EL_BP_100_ncd',
                   'S_HET_1_cds', 'S_HET_1_utr', 'S_HET_1_intron', 
                    'S_HET_10_cds', 'S_HET_10_utr', 'S_HET_10_intron', 
                    'S_HET_100_cds', 'S_HET_100_utr', 'S_HET_100_intron']
consts_4_subsec2 = ['EV_CONS_EL_FRAC_2_cds', 'EV_CONS_EL_FRAC_2_utr', 'EV_CONS_EL_FRAC_2_ncd', 
                   'EV_CONS_EL_FRAC_5_cds', 'EV_CONS_EL_FRAC_5_utr', 'EV_CONS_EL_FRAC_5_ncd', 
                   'EV_CONS_EL_FRAC_10_cds', 'EV_CONS_EL_FRAC_10_utr', 'EV_CONS_EL_FRAC_10_ncd']
consts_subsec_1 = ['PLI_cds', 'PLI_utr', 'PLI_intron', 
                   'LOEUF_cds', 'LOEUF_utr', 'LOEUF_intron', 
                   'genic_cds', 'genic_utr', 'genic_intron']
consts_subsec_2 = ['FDR_ASD_cds', 'FDR_ASD_utr', 'FDR_ASD_intron', 
                   'FDR_DD_cds', 'FDR_DD_utr', 'FDR_DD_intron', 
                   'FDR_NDD_cds', 'FDR_NDD_utr', 'FDR_NDD_intron']
consts_subsec_3 = ['PLIp9_cds', 'PLIp9_utr', 'PLIp9_intron',
                   'LOWPLI_cds', 'LOWPLI_utr', 'LOWPLI_intron']
dev_br_10 = ['DEV_BR_GENE_10', 'DEV_BR_GENE_10_cds', 'DEV_BR_GENE_10_utr', 'DEV_BR_GENE_10_intron']
dev_br_20 = ['DEV_BR_GENE_20', 'DEV_BR_GENE_20_cds', 'DEV_BR_GENE_20_utr', 'DEV_BR_GENE_20_intron']
dev_br_30 = ['DEV_BR_GENE_30', 'DEV_BR_GENE_30_cds', 'DEV_BR_GENE_30_utr', 'DEV_BR_GENE_30_intron']
dev_br_40 = ['DEV_BR_GENE_40', 'DEV_BR_GENE_40_cds', 'DEV_BR_GENE_40_utr', 'DEV_BR_GENE_40_intron']
dev_br_50 = ['DEV_BR_GENE_50', 'DEV_BR_GENE_50_cds', 'DEV_BR_GENE_50_utr', 'DEV_BR_GENE_50_intron']
lof_dev_br = ['LOF_DEV_BR_50_cds', 'LOF_DEV_BR_40_cds', 'LOF_DEV_BR_30_cds', 'LOF_DEV_BR_20_cds', 'LOF_DEV_BR_10_cds']
lof_plus_consts = ['LOF_DEV_BR_10_cds', 'LOF_PLI_cds', 'PLI_cds', 'PLI_utr', 'PLI_intron',
                   'LOF_LOEUF_cds', 'LOEUF_cds', 'LOEUF_utr', 'LOEUF_intron']
lof_plus_consts2 = ['LOF_DEV_BR_10_cds', 'LOF_PLI_cds', 'PLI_cds', 'PLI_utr', 'PLI_intron',
                   'LOF_LOWPLI_cds',  'LOWPLI_cds', 'LOWPLI_utr', 'LOWPLI_intron']
lof_plus_consts3 = ['LOF_PLIp9_cds', 'PLIp9_cds', 'PLIp9_utr', 'PLIp9_intron']
lof_plus_consts4 = ['LOF_FDR_ASD_cds', 'LOF_FDR_DD_cds', 'LOF_FDR_NDD_cds']
lof_plus_consts5 = ['PLIp9_LOF_DEV_BR_10_cds', 
                    'PLIp9_DEV_BR_GENE_10', 
                    'PLIp9_DEV_BR_GENE_10_cds', 'PLIp9_DEV_BR_GENE_10_utr', 'PLIp9_DEV_BR_GENE_10_intron']

In [16]:
### write counts for each sample for R script
# with ins/dup and del separated
# add genome wide burden with ins/dup and del stratification

fts = (consts_1 + consts_2 + consts_3 + consts_3_subsec + consts_subsec_1 + consts_subsec_2 + 
       dev_br_10 + dev_br_20 + dev_br_30 + dev_br_40 + dev_br_50 + lof_dev_br + 
       lof_plus_consts + lof_plus_consts2 + lof_plus_consts3 + lof_plus_consts4 + lof_plus_consts5)

#flt_names = ['AD5_SAMPLES', 'AD4_SAMPLES', 'AD3_SAMPLES', 
#             'SQ20_SAMPLES', 'SQ30_SAMPLES', 'SQ40_SAMPLES', 'SQ50_SAMPLES', 'SQ60_SAMPLES', 'SQ70_SAMPLES']
flt_names = ['SQ20_SAMPLES']

include_svtypes = ['INS', 'DEL', 'DUP', 'INV', '.']
include_svtypes_del = ['DEL']
include_svtypes_ins = ['INS', 'DUP', '.']

samples_col = 'AD2_SAMPLES'

freq_col = 'AD2_SAMPLES'

for flt_name in flt_names:
    print(flt_name)
    # filter for main svtypes and quality
    df_flt = get_df_flt(df, flt_name, include_svtypes, flt_sd_art=True, freq_col=freq_col, freq_thr=0.05)
    print(f'total: {df.shape}')
    print(f'filtered: {df_flt.shape}')
    df_main = pd.DataFrame(df_pca_flt)
    ############## compute genomewide burden ##############
    ### using IL genotypes for LR_IL calls
    temp = (df_flt.loc[df_flt.PLATFORM_CV=='LR'][samples_col].str.split(',').tolist() +
            df_flt.loc[df_flt.PLATFORM_CV=='LR_IL']['IL_GT_SAMPLES_LR'].str.split(',').tolist() + 
            df_flt.loc[df_flt.PLATFORM_CV=='LR_IL']['IL_SAMPLES_LR'].str.split(',').tolist() + 
            df_flt.loc[df_flt.PLATFORM_CV=='IL']['IL_SAMPLES_LR'].str.split(',').tolist() +
            df_flt.loc[df_flt.PLATFORM_CV=='LR_IL']['SV2_GT_SAMPLES'].str.split(',').tolist())
    ### filtering LR-only calls all together
    temp_del = df_flt.loc[df_flt.SVTYPE.isin(include_svtypes_del)][samples_col].str.split(',').tolist() + df_flt.loc[df_flt.SVTYPE.isin(include_svtypes_del)]['IL_SAMPLES_LR'].str.split(',').tolist()
    temp_ins = df_flt.loc[df_flt.SVTYPE.isin(include_svtypes_ins)][samples_col].str.split(',').tolist() + df_flt.loc[df_flt.SVTYPE.isin(include_svtypes_ins)]['IL_SAMPLES_LR'].str.split(',').tolist()
    temp_lr = df_flt.loc[df_flt.PLATFORM_CV=='LR'][samples_col].str.split(',').tolist()
    temp_lr_il_lrgt = df_flt.loc[df_flt.PLATFORM_CV=='LR_IL'][samples_col].str.split(',').tolist()
    ### IL_GT_SAMPLES_LR: for calls coming from LR_IL, IL_SAMPLES_LR: for calls coming from IL, they shouldn't be both present
    temp_lr_il_ilgt = (df_flt.loc[df_flt.PLATFORM_CV=='LR_IL']['IL_GT_SAMPLES_LR'].str.split(',').tolist() + 
                       df_flt.loc[df_flt.PLATFORM_CV=='LR_IL']['IL_SAMPLES_LR'].str.split(',').tolist() + 
                       df_flt.loc[df_flt.PLATFORM_CV=='LR_IL']['SV2_GT_SAMPLES'].str.split(',').tolist())
    temp_il = df_flt.loc[df_flt.PLATFORM_CV=='IL']['IL_SAMPLES_LR'].str.split(',').tolist()
    this_samples_list = []
    this_samples_del_list = []
    this_samples_ins_list = []
    this_samples_lr_list = []
    this_samples_lr_il_lrgt_list = []
    this_samples_lr_il_ilgt_list = []
    this_samples_il_list = []
    for x in temp:
        this_samples_list.extend(x)
    for x in temp_del:
        this_samples_del_list.extend(x)
    for x in temp_ins:
        this_samples_ins_list.extend(x)
    for x in temp_lr:
        this_samples_lr_list.extend(x)
    for x in temp_lr_il_lrgt:
        this_samples_lr_il_lrgt_list.extend(x)
    for x in temp_lr_il_ilgt:
        this_samples_lr_il_ilgt_list.extend(x)
    for x in temp_il:
        this_samples_il_list.extend(x)
    count_col = []
    count_col_del = []
    count_col_ins = []
    count_col_lr = []
    count_col_lr_il_lrgt = []
    count_col_lr_il_ilgt = []
    count_col_il = []
    for sample in df_pca_flt.iid.tolist():
        count_col.append(this_samples_list.count(sample))
        count_col_del.append(this_samples_del_list.count(sample))
        count_col_ins.append(this_samples_ins_list.count(sample))
        count_col_lr.append(this_samples_lr_list.count(sample))
        count_col_lr_il_lrgt.append(this_samples_lr_il_lrgt_list.count(sample))
        count_col_lr_il_ilgt.append(this_samples_lr_il_ilgt_list.count(sample))
        count_col_il.append(this_samples_il_list.count(sample))
    df_main[f'count_genomewide'] = count_col
    df_main[f'count_genomewide_del'] = count_col_del
    df_main[f'count_genomewide_ins'] = count_col_ins
    df_main[f'count_genomewide_lr'] = count_col_lr
    df_main[f'count_genomewide_lr_il_lrgt'] = count_col_lr_il_lrgt
    df_main[f'count_genomewide_lr_il_ilgt'] = count_col_lr_il_ilgt
    df_main[f'count_genomewide_il'] = count_col_il
    #######################################################
    for ft in fts:
        #print(ft)
        this_df = df_flt.loc[df_flt[f'X_{ft}'] == 1]
        ### using IL genotypes for LR_IL calls
        temp = (this_df.loc[this_df.PLATFORM_CV=='LR'][samples_col].str.split(',').tolist() +
                this_df.loc[this_df.PLATFORM_CV=='LR_IL']['IL_GT_SAMPLES_LR'].str.split(',').tolist() + 
                this_df.loc[this_df.PLATFORM_CV=='LR_IL']['IL_SAMPLES_LR'].str.split(',').tolist() + 
                this_df.loc[this_df.PLATFORM_CV=='IL']['IL_SAMPLES_LR'].str.split(',').tolist() +
                this_df.loc[this_df.PLATFORM_CV=='LR_IL']['SV2_GT_SAMPLES'].str.split(',').tolist())
        ### filtering LR-only calls all together
        temp_del = this_df.loc[this_df.SVTYPE.isin(include_svtypes_del)][samples_col].str.split(',').tolist() + this_df.loc[this_df.SVTYPE.isin(include_svtypes_del)]['IL_SAMPLES_LR'].str.split(',').tolist()
        temp_ins = this_df.loc[this_df.SVTYPE.isin(include_svtypes_ins)][samples_col].str.split(',').tolist() + this_df.loc[this_df.SVTYPE.isin(include_svtypes_ins)]['IL_SAMPLES_LR'].str.split(',').tolist()
        temp_lr = this_df.loc[this_df.PLATFORM_CV=='LR'][samples_col].str.split(',').tolist()
        temp_lr_il_lrgt = this_df.loc[this_df.PLATFORM_CV=='LR_IL'][samples_col].str.split(',').tolist()
        ### IL_GT_SAMPLES_LR: for calls coming from LR_IL, IL_SAMPLES_LR: for calls coming from IL, they shouldn't be both present
        temp_lr_il_ilgt = (this_df.loc[this_df.PLATFORM_CV=='LR_IL']['IL_GT_SAMPLES_LR'].str.split(',').tolist() + 
                           this_df.loc[this_df.PLATFORM_CV=='LR_IL']['IL_SAMPLES_LR'].str.split(',').tolist() + 
                           this_df.loc[this_df.PLATFORM_CV=='LR_IL']['SV2_GT_SAMPLES'].str.split(',').tolist())
        temp_il = this_df.loc[this_df.PLATFORM_CV=='IL']['IL_SAMPLES_LR'].str.split(',').tolist()
        this_samples_list = []
        this_samples_del_list = []
        this_samples_ins_list = []
        this_samples_lr_list = []
        this_samples_lr_il_lrgt_list = []
        this_samples_lr_il_ilgt_list = []
        this_samples_il_list = []
        for x in temp:
            this_samples_list.extend(x)
        for x in temp_del:
            this_samples_del_list.extend(x)
        for x in temp_ins:
            this_samples_ins_list.extend(x)
        for x in temp_lr:
            this_samples_lr_list.extend(x)
        for x in temp_lr_il_lrgt:
            this_samples_lr_il_lrgt_list.extend(x)
        for x in temp_lr_il_ilgt:
            this_samples_lr_il_ilgt_list.extend(x)
        for x in temp_il:
            this_samples_il_list.extend(x)
        count_col = []
        count_col_del = []
        count_col_ins = []
        count_col_lr = []
        count_col_lr_il_lrgt = []
        count_col_lr_il_ilgt = []
        count_col_il = []
        for sample in df_pca_flt.iid.tolist():
            count_col.append(this_samples_list.count(sample))
            count_col_del.append(this_samples_del_list.count(sample))
            count_col_ins.append(this_samples_ins_list.count(sample))
            count_col_lr.append(this_samples_lr_list.count(sample))
            count_col_lr_il_lrgt.append(this_samples_lr_il_lrgt_list.count(sample))
            count_col_lr_il_ilgt.append(this_samples_lr_il_ilgt_list.count(sample))
            count_col_il.append(this_samples_il_list.count(sample))
        df_main[f'count_{ft}'] = count_col
        df_main[f'count_{ft}_del'] = count_col_del
        df_main[f'count_{ft}_ins'] = count_col_ins
        df_main[f'count_{ft}_lr'] = count_col_lr
        df_main[f'count_{ft}_lr_il_lrgt'] = count_col_lr_il_lrgt
        df_main[f'count_{ft}_lr_il_ilgt'] = count_col_lr_il_ilgt
        df_main[f'count_{ft}_il'] = count_col_il

    #display(df_main)
    file_out = f'{dir_out}/main_df_toR_freq05_LRIL_{flt_name}_{samples_col}_DELINS_GW_plat.csv'
    print(f'writing {file_out}')
    df_main.to_csv(file_out, sep=',', header=True, index=False)
print('+'*40)

SQ20_SAMPLES
total: (344746, 434)
filtered: (27279, 434)
writing outputs//main_df_toR_freq05_LRIL_SQ20_SAMPLES_AD2_SAMPLES_DELINS_GW_plat.csv
++++++++++++++++++++++++++++++++++++++++
